In [69]:
import numpy as np
import pandas as pd
import re

myrankings = pd.DataFrame(pd.read_csv('myrankings.csv'))
myrankings.drop(columns=["Team", "ESPN Ranking"], inplace=True)
myrankings.dropna(inplace=True)
underdog_rankings = pd.DataFrame(pd.read_csv('underdog_rankings.csv'))

# Step 1: Merge
merged = pd.merge(myrankings, underdog_rankings, on='Player', how='outer', suffixes=('_my', '_underdog'))

# Step 2: Create a rank key for sorting
# First, create a mapping of player → index in myrankings
myrank_index = {player: i for i, player in enumerate(myrankings['Player'])}
underdog_index = {player: i for i, player in enumerate(underdog_rankings['Player'])}

# Step 3: Apply a combined sort key
def custom_sort_key(row):
    player = row['Player']
    if player in myrank_index:
        return myrank_index[player]  # Prioritize myrankings order
    else:
        return 10000 + underdog_index.get(player, 9999)  # Put the rest after

merged['sort_key'] = merged.apply(custom_sort_key, axis=1)

# Step 4: Sort and clean
merged_sorted = merged.sort_values(by='sort_key').drop(columns='sort_key').reset_index(drop=True)

merged_sorted.drop(columns=["Rank", "ADP_my", "Position", "My Pos Rank", "Id"], inplace=True)
merged_sorted.rename(columns={"ADP_underdog": "ADP"}, inplace=True)
merged_sorted["Rank"] = [i for i in range(1, len(merged_sorted)+1)]

merged_sorted = merged_sorted[['Rank', 'Player', 'Pos', 'Finish2024', 'Notes']]


In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Find the main table
table = soup.find("table", {"id": "data"})

# Convert HTML table to DataFrame
df = pd.read_html(str(table))[0]

# Clean column names if needed
df.columns = [col.strip() for col in df.columns]

pattern = r"^(.*?)\s+([A-Z]{2,3})\s+\((\d+)\)$"

df[['Player', 'Team', 'Bye']] = df['Player Team (Bye)'].str.extract(pattern)

df.drop(columns=['Player Team (Bye)'], inplace=True)

df[["Player", "Team", "Bye"]] = df[["Player", "Team", "Bye"]].apply(lambda x: x.str.strip())

df = df[["Player", "Team", "Bye", "AVG"]]

df.to_csv("adp.csv", index=False)



C:\Users\tyler\AppData\Local\Temp\ipykernel_25388\3084821536.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [71]:

def clean_name(name):
    name = name.lower()                            # Lowercase
    name = name.replace('.', '')                   # Remove dots
    name = re.sub(r'\b(sr|jr|ii|iii)\b', '', name)  # Remove suffixes
    name = re.sub(r'\s+', ' ', name)               # Collapse multiple spaces
    return name.strip()

merged_sorted['Player_clean'] = merged_sorted['Player'].apply(clean_name)
df['Player_clean'] = df['Player'].astype(str).apply(clean_name)

rankings = pd.merge(merged_sorted, df, on='Player_clean', how='inner', suffixes=('_merged', '_df'))
rankings.drop(columns=['Player_clean', 'Player_df'], inplace=True)
rankings.rename(columns={"Player_merged": "Player"}, inplace=True)
rankings["Pos Rank"] = rankings.groupby('Pos').cumcount() + 1

rankings.rename(columns={"AVG": "ADP"}, inplace=True)
rankings = rankings[['Rank', 'Player', 'Pos', 'Team', 'ADP', 'Pos Rank', 'Bye', 'Finish2024', 'Notes']]

rankings['sort_key'] = rankings.apply(
     lambda row: (row['Rank'], row['ADP']) if pd.notna(row['ADP']) else (10000, row['Rank']),
            axis=1
)

# Step 4: Sort and clean
rankings_sorted=rankings.sort_values(by='sort_key').drop(columns='sort_key').reset_index(drop=True)

rankings_sorted.to_csv('updated_rankings.csv', index=False)
